# Assignment Solutions

In [1]:
# imports
from torchvision.datasets import MNIST
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Question 1
mnist_data_train = MNIST('/', train=True, download=True)
mnist_data_test = MNIST('/', train=False, download=True)

In [20]:
# Question 3
import numpy as np

# Step 1: Generate random vectors p and q
p = np.random.uniform(-10, 10, 100)
q = np.random.uniform(-10, 10, 100)

# Step 2: Generate target variable b based on specified conditions
b = np.where(p * q > 1, 1, -1)

# Step 3: Define basis functions
def f0(p, q):
    return np.ones_like(p)

def f1(p, q):
    return p

def f2(p, q):
    return q

def f3(p, q):
    return p ** 2

def f4(p, q):
    return q ** 2

def f5(p, q):
    return p * q

# Step 4: Create the matrix A using the basis functions
A = np.column_stack([f0(p, q), f1(p, q), f2(p, q), f3(p, q), f4(p, q), f5(p, q)])

# Step 5: Use least squares to find the coefficient vector x
x = np.linalg.lstsq(A, b, rcond=None)[0]

# Step 6: Compute the predicted values
y_pred = np.dot(A, x)
# print(y_pred)
# print(b)

# Step 7: Compute the mean squared error
mse = np.mean((b - y_pred) ** 2)
print(mse)

0.36747535219018695
